In [1]:
import pymorphy2
import pandas as pd
from tqdm import tqdm
import string
import numpy as np
import faiss
import os
import fasttext


In [2]:
dataset = pd.read_csv('../dataset_2.csv', chunksize=100)

In [3]:
dataset = next(iter(dataset))

In [4]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/appuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
tqdm.pandas()


class Searcher():
    def __init__(self, data, search_type, b=0.75, k=2, texts=None, inv_index=None):
        self.morph = pymorphy2.MorphAnalyzer()
        self.punctuation = string.punctuation + '…' + '«' + '»' + '—'
        self.stop_words = stopwords.words('russian')
        self.use_lemmatization = search_type == 'bm25_lemm'
        self.search_type = search_type

        self.vectorizer = fasttext.load_model("/home/appuser/infosearch/cc.ru.300.bin") if search_type == 'vec' else None

        self.k = k
        self.b = b

        self.data = data # исходные данные

        if texts is None:
            print('Preprocessing texts...')
            if search_type != 'vec':
                self.texts = data.text.progress_apply(self.preprocess_text) # очищенные тексты в нормальной форме
            else:
                self.texts = data.text
        else:
            self.texts = texts

        if inv_index is None:
            print('Counting inverted index...')
            
            if search_type != 'vec':
                inv_index = {}

                for doc_id, doc in enumerate(tqdm(self.texts)):
                    tokens = doc.split()
                    for token in tokens:
                        if token in inv_index:
                            inv_index[token].add(doc_id)
                        else:
                            inv_index[token] = set([doc_id])

            else:
                texts_vectorized = np.zeros([len(self.texts), self.vectorizer.get_dimension()])
                for doc_id, doc in enumerate(tqdm(self.texts)):
                    texts_vectorized[doc_id] = self.vectorizer[str(doc)]

                inv_index = faiss.IndexFlatL2(texts_vectorized.shape[1])
                inv_index.add(texts_vectorized)

                faiss.write_index(inv_index, os.path.join('/home/appuser/infosearch', "vec.index"))
        
        self.inv_index = inv_index

    def preprocess_text(self, text):
        tokens = str(text).lower().translate(str.maketrans('', '', self.punctuation)).split()
        res = ''

        if self.use_lemmatization:
            for token in tokens:
                p = self.morph.parse(token)[0].normal_form
                if p not in self.stop_words:
                    res += ' ' + p
        else:
            for token in tokens:
                if token not in self.stop_words:
                    res += ' ' + token
        
        return res


    def idf(self, terms, documents, eps=-0.01):
        idfs = np.asarray([len(documents) / (len(self.inv_index[term]) + eps) for term in terms])
        return np.log(idfs)


    def tf(self, terms, doc_ids):
        # можно посплитить текст в init и не сплитить каждый раз
        def count(text):
            freqs = dict.fromkeys(terms, 0)
            text = text.split() 
            for term in text:
                if term in freqs.keys():
                    freqs[term] += 1
            
            return np.asarray([term_freq[1] for term_freq in freqs.items()]) / len(text)

        docs = self.texts[doc_ids]

        return np.stack(docs.apply(count).values)
    
    def tf_idf(self, terms, documents):
        tf_scores = self.tf(terms, documents)
        idf_scores = self.idf(terms, documents)

        scores = tf_scores * idf_scores
        return scores

    def bm25(self, terms, documents):
        
        tf_scores = self.tf(terms, documents)
        idf_scores = self.idf(terms, documents)
        lens = self.texts[documents].apply(lambda x: len(x.split())).to_numpy()

        scores = np.zeros(lens.shape)
        for i, term in enumerate(terms):
            a = (tf_scores[:, i] * (self.k + 1))
            b = (tf_scores[:, i] + self.k * (1 - self.b + self.b * lens / lens.mean()))
            c = idf_scores[i]
            scores += a / b * c

        return scores

    def search(self, query):
        if self.search_type != 'vec':
            terms = self.preprocess_text(query).split()

            global_ids = set()
            for term in terms:
                global_ids.update(self.inv_index[term])

            global_ids = np.asarray(list(global_ids))
            scores = self.bm25(terms, global_ids)#.sum(axis=1)

            local_ids = np.argsort(scores, axis=0).squeeze() # сортируем документы по возрастанию tf-idf
            

            return self.data.iloc[global_ids[local_ids][-10:]]
        
        else:
            query_vector = self.vectorizer[query]

            distances, indicies = self.inv_index.search(np.expand_dims(query_vector, 0), 10)

            print(distances)
            print(indicies[0])

            return self.data.iloc[indicies[0]]
        
        

In [6]:
s = Searcher(dataset, search_type='vec')

Preprocessing texts...
Counting inverted index...


100%|██████████| 100/100 [00:01<00:00, 97.22it/s]


In [7]:
s.search('быдло')

[[1.2019951 1.2022532 1.2029874 1.2032571 1.2037507 1.2041138 1.2043403
  1.2049005 1.205671  1.2059832]]
[52 16 37  7 88 85 89 92 32  9]


,Unnamed: 0,title,link,text
52,52,Педокомпания,/wiki/%D0%9F%D0%B5%D0%B4%D0%BE%D0%BA%D0%BE%D0%...,Педокомпания (англ. Pedopals) — создана на 7ch...
16,16,Расово,/wiki/%D0%A0%D0%B0%D1%81%D0%BE%D0%B2%D0%BE,Расово (moon. 人種的に)— одно из любимых изречений...
37,37,Бобёр-извращенец,/wiki/%D0%91%D0%BE%D0%B1%D1%91%D1%80-%D0%B8%D0...,В реку случился несанкционированный выброс виа...
7,7,Поцреот,/wiki/%D0%9F%D0%BE%D1%86%D1%80%D0%B5%D0%BE%D1%82,мы… рода Рускаго: Карлы Инегелдъ Фарлофъ Верем...
88,88,Довольно слабый петух,/wiki/%D0%94%D0%BE%D0%B2%D0%BE%D0%BB%D1%8C%D0%...,Довольно слабый петух — характеристика для пет...
85,85,Похрюкивалка,/wiki/%D0%9F%D0%BE%D1%85%D1%80%D1%8E%D0%BA%D0%...,Похрюкивалка — распространённое название для с...
89,89,Лев,/wiki/%D0%9B%D0%B5%D0%B2,Лев — мощное рычало котовой направленности; де...
92,92,Потявкун,/wiki/%D0%9F%D0%BE%D1%82%D1%8F%D0%B2%D0%BA%D1%...,Потявкун (Погавкун) — народное название для со...
32,32,Жывотнайе,/wiki/%D0%96%D1%8B%D0%B2%D0%BE%D1%82%D0%BD%D0%...,"Жывотнайе (жыготнайе, жыготное, жывтоне) — низ..."
9,9,Быдло,/wiki/%D0%91%D1%8B%D0%B4%D0%BB%D0%BE,Будь проще — шути больше— Заветы всего быдла ...
